In [3]:
import cv2
import os
import numpy as np
import pandas as pd
from datetime import datetime
from tkinter import Tk, Label, Entry, Button, filedialog
import shutil


In [4]:
# Constants
DATASET_PATH = 'dataset'
TRAINER_PATH = 'trainer'
MODEL_FILE = os.path.join(TRAINER_PATH, 'trainer.yml')
ATTENDANCE_FILE = 'attendance.csv'

# Ensure necessary directories exist
os.makedirs(DATASET_PATH, exist_ok=True)
os.makedirs(TRAINER_PATH, exist_ok=True)

# Initialize Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [5]:
class CSVAttendanceSystem:
    def __init__(self, dataset_path=DATASET_PATH, model_file=MODEL_FILE, attendance_file=ATTENDANCE_FILE):
        self.dataset_path = dataset_path
        self.model_file = model_file
        self.attendance_file = attendance_file
        self.face_cascade = face_cascade
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        self.labels = {}

    def collect_training_data(self):
        faces = []
        ids = []
        label_id = 0
        self.labels = {}
        for person_name in os.listdir(self.dataset_path):
            person_dir = os.path.join(self.dataset_path, person_name)
            if os.path.isdir(person_dir):
                self.labels[label_id] = person_name
                for img_file in os.listdir(person_dir):
                    img_path = os.path.join(person_dir, img_file)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    if img is None:
                        continue
                    img = cv2.resize(img, (100, 100))
                    faces.append(img)
                    ids.append(label_id)
                label_id += 1
        return faces, ids

    def train_model(self):
        print("[INFO] Training model...")
        faces, ids = self.collect_training_data()
        if len(faces) == 0:
            print("[ERROR] No training data found.")
            return
        self.recognizer.train(faces, np.array(ids))
        self.recognizer.save(self.model_file)
        print(f"[INFO] Model trained with {len(faces)} images.")

    def load_model(self):
        if os.path.exists(self.model_file):
            self.recognizer.read(self.model_file)
            print("[INFO] Model loaded successfully.")
        else:
            print("[ERROR] Model file not found.")

    def mark_attendance(self, name):
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        df = pd.DataFrame([[name, now]], columns=['Name', 'Time'])
        if os.path.exists(self.attendance_file):
            df.to_csv(self.attendance_file, mode='a', header=False, index=False)
        else:
            df.to_csv(self.attendance_file, index=False)

    def run_real_time_attendance(self):
        self.load_model()
        cap = cv2.VideoCapture(0)
        print("[INFO] Starting webcam for real-time attendance. Press 'q' to quit.")
        recognized = set()
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)

                for (x, y, w, h) in faces:
                    face = gray[y:y+h, x:x+w]
                    face = cv2.resize(face, (100, 100))
                    label_id, confidence = self.recognizer.predict(face)

                    if confidence < 80:
                        name = self.labels.get(label_id, "Unknown")
                        if name not in recognized:
                            self.mark_attendance(name)
                            recognized.add(name)

                        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                        cv2.putText(frame, f"{name} ({int(confidence)})", (x, y-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                cv2.imshow("Attendance System", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        finally:
            cap.release()
            cv2.destroyAllWindows()


In [6]:
def create_dataset_from_webcam(dataset_path, name, num_samples=10, delay=1.0, update_callback=None):
    person_dir = os.path.join(dataset_path, name)
    os.makedirs(person_dir, exist_ok=True)
    cap = cv2.VideoCapture(0)
    count = 0
    print(f"[INFO] Capturing images for {name}...")
    try:
        while count < num_samples:
            ret, frame = cap.read()
            if not ret:
                continue
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                face = gray[y:y+h, x:x+w]
                face = cv2.resize(face, (100, 100))
                file_path = os.path.join(person_dir, f"{count}.jpg")
                cv2.imwrite(file_path, face)
                count += 1
                if update_callback:
                    update_callback(f"Samples captured: {count}")
                cv2.imshow("Capturing...", frame)
                cv2.waitKey(int(delay * 1000))
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()


In [12]:
def add_external_images(system, dataset_path, person_name):
    file_paths = filedialog.askopenfilenames(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    
    person_dir = os.path.join(dataset_path, person_name)
    os.makedirs(person_dir, exist_ok=True)

    count = len(os.listdir(person_dir))
    for file_path in file_paths:
        img = cv2.imread(file_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = system.face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            print(f"[WARNING] No face detected in {os.path.basename(file_path)}. Skipping.")
            continue

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (100, 100))
            img_name = os.path.join(person_dir, f"{count}.jpg")
            cv2.imwrite(img_name, face_resized)
            print(f"[INFO] Saved processed image: {img_name}")
            count += 1
            break  # Process only the first detected face


In [13]:
def preprocess_and_store_external_images(dataset_path, name):
    person_dir = os.path.join(dataset_path, name)
    for file in os.listdir(person_dir):
        path = os.path.join(person_dir, file)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        resized = cv2.resize(img, (100, 100))
        cv2.imwrite(path, resized)  # Overwrite or optionally store in another dir


In [ ]:
def start_gui(system):
    def capture_faces():
        name = name_entry.get().strip()
        if name:
            create_dataset_from_webcam(DATASET_PATH, name, num_samples=10, delay=1.0)
            print(f"[INFO] Captured face images for: {name}")

    def add_external_faces():
        name = name_entry.get().strip()
        if name:
            add_external_images(DATASET_PATH, name)
            preprocess_and_store_external_images(DATASET_PATH, name)
            print(f"[INFO] Added external images for: {name}")

    def train_model():
        system.train_model()

    def start_attendance():
        system.run_real_time_attendance()

    root = Tk()
    root.title("Face Attendance System")

    Label(root, text="Enter Name:").grid(row=0, column=0, padx=10, pady=10)
    name_entry = Entry(root)
    name_entry.grid(row=0, column=1, padx=10, pady=10)

    Button(root, text="Capture via Webcam", command=capture_faces).grid(row=1, column=0, padx=10, pady=10)
    Button(root, text="Add External Images", command=lambda: add_external_images(attendance_system, 'dataset', name_entry.get())).pack()
    Button(root, text="Train Model", command=train_model).grid(row=2, column=0, padx=10, pady=10)
    Button(root, text="Start Attendance", command=start_attendance).grid(row=2, column=1, padx=10, pady=10)

    root.mainloop()


In [11]:
# Initialize the system
attendance_system = CSVAttendanceSystem()

# Start the GUI
start_gui(attendance_system)


[INFO] Added external image: dataset\Chirag\0.jpg
[INFO] Added external image: dataset\Chirag\1.jpg
[INFO] Added external image: dataset\Chirag\2.jpg
[INFO] Added external images for: Chirag
[ERROR] Model file not found.
[INFO] Starting webcam for real-time attendance. Press 'q' to quit.


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\tanya\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\tanya\AppData\Local\Temp\ipykernel_27884\2639756711.py", line 19, in start_attendance
    system.run_real_time_attendance()
  File "C:\Users\tanya\AppData\Local\Temp\ipykernel_27884\1505704701.py", line 71, in run_real_time_attendance
    label_id, confidence = self.recognizer.predict(face)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\face\src\lbph_faces.cpp:406: error: (-5:Bad argument) This LBPH model is not computed yet. Did you call the train method? in function 'cv::face::LBPH::predict'



[INFO] Training model...
[INFO] Model trained with 3 images.
[INFO] Model loaded successfully.
[INFO] Starting webcam for real-time attendance. Press 'q' to quit.
[INFO] Capturing images for Chaitanya...
[INFO] Captured face images for: Chaitanya
[INFO] Training model...
[INFO] Model trained with 13 images.
[INFO] Model loaded successfully.
[INFO] Starting webcam for real-time attendance. Press 'q' to quit.
